# Simpsons_001

* v001: initial commit

In [1]:
from fastai.imports import *
from os.path import join
from torchtext import vocab, data
from fastai.conv_learner import *

In [3]:
from fastai.nlp import *
from fastai.lm_rnn import *

In [4]:
PATH='./'

TRN_PATH = 'trn/'
VAL_PATH = 'val/'
TRN = f'{PATH}{TRN_PATH}'
VAL = f'{PATH}{VAL_PATH}'

# Note: The student needs to practice her shell skills and prepare her own dataset before proceeding:
# - trn/trn.txt (first 80% of nietzsche.txt)
# - val/val.txt (last 20% of nietzsche.txt)

%ls {PATH}

all.txt    Simpsons_get dialogues_v001.ipynb  trn/   val/
README.md  Simpsons_v001.ipynb                txts/


In [5]:
filename = "all.txt"
dialogues = open(join(PATH,filename),'r', encoding='latin1').read()

In [6]:
trn = dialogues[:int(len(dialogues)*.8)]
val = dialogues[int(len(dialogues)*.8):]
with open(join(TRN, 'trn.txt'), 'w') as f:
    f.write(trn)
with open(join(VAL, 'val.txt'), 'w') as f:
    f.write(val)


In [7]:
TEXT = data.Field(lower=True, tokenize=list)
bs=64; bptt=8; n_fac=42; n_hidden=256

FILES = dict(train=TRN_PATH, validation=VAL_PATH, test=VAL_PATH)
md = LanguageModelData.from_text_files(PATH, TEXT, **FILES, bs=bs, bptt=bptt, min_freq=3)

len(md.trn_dl), md.nt, len(md.trn_ds), len(md.trn_ds[0].text)

(13315, 77, 1, 6818033)

In [8]:
class CharSeqStatefulGRU(nn.Module):
    def __init__(self, vocab_size, n_fac, bs):
        super().__init__()
        self.vocab_size = vocab_size
        self.e = nn.Embedding(vocab_size, n_fac)
        self.rnn = nn.GRU(n_fac, n_hidden)
        self.l_out = nn.Linear(n_hidden, vocab_size)
        self.init_hidden(bs)
        
    def forward(self, cs):
        bs = cs[0].size(0)
        if self.h.size(1) != bs: self.init_hidden(bs)
        outp,h = self.rnn(self.e(cs), self.h)
        self.h = repackage_var(h)
        return F.log_softmax(self.l_out(outp), dim=-1).view(-1, self.vocab_size)
    
    def init_hidden(self, bs): self.h = V(torch.zeros(1, bs, n_hidden))

In [10]:
m = CharSeqStatefulGRU(md.nt, n_fac, 512).cpu()

opt = optim.Adam(m.parameters(), 1e-3)

In [11]:
fit(m, md, 6, opt, F.nll_loss)

epoch      trn_loss   val_loss                                  
    0      1.416642   1.511298  
    1      1.372844   1.475769                                  
    2      1.353771   1.463523                                  
    3      1.348616   1.457748                                  
    4      1.337046   1.449532                                  
    5      1.337887   1.446881                                  



[array([1.44688])]

Save model

In [52]:
torch.save(m.state_dict(), join('out', 'model_dict.pt'))
torch.save(m, join('out', 'model.pt'))

/home/lorenzo/.local/lib/python3.6/site-packages/torch/serialization.py:159: UserWarning: Couldn't retrieve source code for container of type CharSeqStatefulGRU. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


Load model

In [54]:
the_model = CharSeqStatefulGRU(md.nt, n_fac, 512)
the_model.load_state_dict(torch.load(join('out', 'model_dict.pt')))

In [60]:
the_model = torch.load(join('out', 'model.pt'))

In [40]:
set_lrs(opt, 1e-4)

In [ ]:
#fit(m, md, 3, opt, F.nll_loss)

In [23]:
def get_next(inp):
    idxs = TEXT.numericalize(inp, device=-1)
    p = m(VV(idxs.transpose(0,1)))
    r = torch.multinomial(p[-1].exp(), 1)
    return TEXT.vocab.itos[to_np(r)[0]]

def get_next_n(inp, n):
    res = inp
    for i in range(n):
        c = get_next(inp)
        res += c
        inp = inp[1:]+c
    return res

In [39]:
get_next_n('mum ', 100)

'mum inside us. how lowed our with alline. so musline, i this pad, rad-us restan. ambiles name media, wel'